# My answers to dlt workshop homework

In [10]:
import dlt
from dlt.sources.helpers.rest_client import RESTClient
from dlt.sources.helpers.rest_client.paginators import PageNumberPaginator

In [5]:
BASE_URL = "https://us-central1-dlthub-analytics.cloudfunctions.net/data_engineering_zoomcamp_api"

## Question 1: What is your dlt version?

In [6]:
print("dlt version:", dlt.__version__)

dlt version: 1.6.1


## Question 2: How many DuckDB tables were created?

In [ ]:
# Define API resource
@dlt.resource(name="rides")     # Resource name will be used as table name

# Implement automatic pagination
def paginated_getter():
    client = RESTClient(
        base_url=BASE_URL,
        # Pagination strategy: page number-based pagination (1, 2, 3, ...)
        # Start from page 1
        # No total count of pages provided by API, so stop when a page contains no result items
        paginator=PageNumberPaginator(
            base_page=1,   
            total_path=None
        )
    )

    # For each page at API endpoint for retreiving taxi trips data
    for page in client.paginate("data_engineering_zoomcamp_api"):
        yield page  # manage memory

# Define dlt pipeline
pipeline = dlt.pipeline(
    pipeline_name="load_ny_taxi_data",
    destination="duckdb",
    dataset_name="ny_taxi_data"
)

# Load to DuckDB
load_info = pipeline.run(paginated_getter, table_name="rides", write_disposition="replace")
print(load_info)


Pipeline load_ny_taxi_data load step completed in 0.91 seconds
1 load package(s) were loaded to destination duckdb and into dataset ny_taxi_data
The duckdb destination used duckdb:////Users/elijahsutton/Projects/de-zoomcamp-2025/workshop_dlt/homework/load_ny_taxi_data.duckdb location to store data
Load package 1739739142.044773 is LOADED and contains no failed jobs


In [ ]:
import duckdb
import pandas as pd
pd.options.display.max_rows = 100

In [16]:
# A database '<pipeline_name>.duckdb' was created in working directory so just connect to it

# Connect to the DuckDB database
conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

# Set search path to the dataset
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")

# Describe the dataset
conn.sql("DESCRIBE").df()

,database,schema,name,column_names,column_types,temporary
0,load_ny_taxi_data,ny_taxi_data,_dlt_loads,"[load_id, schema_name, status, inserted_at, sc...","[VARCHAR, VARCHAR, BIGINT, TIMESTAMP WITH TIME...",False
1,load_ny_taxi_data,ny_taxi_data,_dlt_pipeline_state,"[version, engine_version, pipeline_name, state...","[BIGINT, BIGINT, VARCHAR, VARCHAR, TIMESTAMP W...",False
2,load_ny_taxi_data,ny_taxi_data,_dlt_version,"[version, engine_version, inserted_at, schema_...","[BIGINT, BIGINT, TIMESTAMP WITH TIME ZONE, VAR...",False
3,load_ny_taxi_data,ny_taxi_data,rides,"[end_lat, end_lon, fare_amt, passenger_count, ...","[DOUBLE, DOUBLE, DOUBLE, BIGINT, VARCHAR, DOUB...",False


## Question 3: Total number of ride records extracted?

In [23]:
rides_df = pipeline.dataset(dataset_type="default").rides.df()

In [ ]:
rides_df.shape  # (num rows, num columns)

(10000, 18)

## Question 4: What is the average trip duration?

In [25]:
with pipeline.sql_client() as client:
    res = client.execute_sql(
            """
            SELECT
            AVG(date_diff('minute', trip_pickup_date_time, trip_dropoff_date_time))
            FROM rides;
            """
        )
    # Prints column values of the first row
    print(res)

[(12.3049,)]
